#### Import Julia packages

In [1]:
# import Pkg
# Pkg.add("JSON3")
# Pkg.add("Interpolations")

In [2]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
using Makie
# using Interpolations

#### Import Custom functions

In [3]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

get_control_int (generic function with 1 method)

#### Read JSON File

In [4]:
width, height, windows, doors, heating_elem = get_room("Boundary.json")

(5, 3, Any[(0.75125, 2.32125, "Left"), (0.73125, 2.32125, "Right")], Any[(0.235, 0.655, "Top")], [[2.005, 3.435, 0.10125, 0.20125], [1.765, 3.215, 2.80125, 2.92125]])

In [5]:
price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")

(Any[0.11781, 0.11106, 0.10592, 0.10341, 0.10836, 0.10655, 0.16935, 0.43496, 0.39644], Any[1.38209, 1.3029, 1.2426, 1.21315, 1.27123, 1.24999, 1.98673, 5.10273, 4.65084], Any[282.15, 282.34999999999997, 282.25, 282.34999999999997, 282.15, 281.45, 280.65, 280.84999999999997, 280.75], Any[10, 11, 12, 13, 14, 15, 16, 17, 18])

#### Boundary conditions

In [6]:
f_Γ_w = generate_f_Γ(windows, width, height)
f_Γ_d = generate_f_Γ(doors, width, height)
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))

f_Γ_wall (generic function with 1 method)

#### Domain & Discretization

In [7]:
domain = (0, width , 0, height)
partition = (25,25)
model = CartesianDiscreteModel(domain,partition)
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

UnconstrainedFESpace()

#### Time parameters

In [8]:
t0 = 0.0    # Start time
tF = 28800   # End time
Δt = 40

40

#### Room parameters

In [9]:
h_wall(x) = 0.22*0.02           # [W/m^2K]
h_window(x) = 1.2*0.02
h_door(x) = 1.0*0.02

ρ(x)=1.225
c(x)=1020.0
k(x)=5#3.5


h = (h_wall, h_window, h_door) 
constants = (c, ρ, k, h) 


Tout = get_temp_function(T_out, tF)

Tini(x)=273.15 + 22.0

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(Tini, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 625
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 11801284990662421406

#### Controll parameter

In [50]:
#q_pos(x) = χ(x[1], 0.21, 1.73) * χ(x[2], 0.13, 0.16)
q_pos = heating_elements(heating_elem)

Q(x,t)=0#150*q_pos(x)*χ(t,6.0,9.0)
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

#### Solver parameters

In [11]:
L2norm(u)=√(Δt*((∑(∫(u[1][2]⋅u[1][2])*dΩ)+∑(∫(u[end][2]⋅u[end][2])*dΩ))/2 + ∑(∑(∫(u[k][2]⋅u[k][2])*dΩ) for k=2:length(u)-1)))
L2skp(u)=Δt*((∑(∫(u[1][2]⋅u[1][2])*dΩ)+∑(∫(u[end][2]⋅u[end][2])*dΩ))/2 + ∑(∑(∫(u[k][2]⋅u[k][2])*dΩ) for k=2:length(u)-1))
get_control_int(qs) = [∑(∫(u)*dΩ) for (t,u) in qs] ./ ∑(∫(q_pos)*dΩ)
get_temperature_int(Ts) = [∑(∫(temp)*dΩ) for (t,temp) in Ts] ./ ∑(∫(1)*dΩ)

Proj(a,b,z) = min(max(a,z),b)
a=0.0
b=400*(∑(∫(q_pos)*dΩ))
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]

Proj (generic function with 2 methods)

In [12]:
get_control_int(qs) = [∑(∫(u)*dΩ) for (t,u) in qs] ./ ∑(∫(q_pos)*dΩ)
get_temperature_int(Ts) = [∑(∫(temp)*dΩ) for (t,temp) in Ts] ./ ∑(∫(1)*dΩ)


function result_plot2(; T_int, Q_int, t0, tF, hour_lst, price_NOK, T_out)
    # time = LinRange(t0, tF, length(T_int))
    time = LinRange(t0, tF, length(T_int)) ./ 3600  # convert from seconds to hours
    time2 = LinRange(t0, tF, length(T_int))

    
    time_price = [price(t) for t in time2]

    fig = CairoMakie.Figure(size=(600, 900), figure_padding=(0, 5, 0, 5))

    # Formatter for temperature axis (Kelvin to Celsius)
    temp_formatter = v -> ["$(round(vi - 273.15, digits=1))" for vi in v]

    # Temperature profile
    ax = CairoMakie.Axis(
        fig[1, 1],
        title = "Temperature profile",
        xlabel = "Time (hours)",
        ylabel = "Temperature (°C)",
        ytickformat = temp_formatter
    )
    lines!(ax, time, T_int, color = :green)

    # Heating output
    ax = CairoMakie.Axis(
        fig[2, 1],
        title = "Optimal Heating Schedule",
        xlabel = "Time (hours)",
        ylabel = "Heating Output (W)"
    )
    lines!(ax, time, Q_int, color = :red)

    # Energy price
    ax = CairoMakie.Axis(
        fig[3, 1],
        title = "Energy price over time",
        xlabel = "Time (hours)",
        ylabel = "Price (NOK)"
    )
    lines!(ax, time, time_price, color = :blue)

    # Outside temperature
    ax = CairoMakie.Axis(
        fig[4, 1],
        title = "Outside Temperature over time",
        xlabel = "Time (hours)",
        ylabel = "Temperature (°C)",
        ytickformat = temp_formatter
    )
    lines!(ax, hour_lst .-10, T_out, color = :orange)

    return fig
end

function plot_costs2(costs)
    fig = CairoMakie.Figure(size=(600, 300))

    ax = CairoMakie.Axis(
        fig[1, 1],
        xlabel = "Iteration",
        ylabel = "Cost",
        yscale = log10  # Make y-axis logarithmic
    )

    lines!(ax, 1:length(costs), costs, color = :purple)

    return fig
end



plot_costs2 (generic function with 1 method)

#### Test Gradient DEs

In [36]:
price = get_price_function(price_NOK, tF, 1)
γ = 1.225*1020*3

(Ts3,qs3,Ws3,costs3) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=30,
armijoparas=(ρ=0.5, α_0=10, α_min=1e-4, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=3.062372600991197e6, k = 1
α = 10, new_cost = 1.0892622906096334e7, L2fgrad = 10389.635550868732, interm = 0.0
α = 5.0, new_cost = 6.205124691313479e6, L2fgrad = 10389.635550868732, interm = 0.0
α = 2.5, new_cost = 944815.062952947, L2fgrad = 10389.635550868732, interm = 0.0
entered for loop, E=944815.062952947, k = 2
α = 10, new_cost = 3.062372600991197e6, L2fgrad = 3666.120803322863, interm = 0.0
α = 5.0, new_cost = 1.4579696193936511e6, L2fgrad = 3666.120803322863, interm = 0.0
α = 2.5, new_cost = 667052.0569364149, L2fgrad = 3666.120803322863, interm = 0.0
entered for loop, E=667052.0569364149, k = 3
α = 10, new_cost = 1.2497949175206302e6, L2fgrad = 1393.2788337114116, interm = 0.0
α = 5.0, new_cost = 696573.6659260283, L2fgrad = 1393.2788337114116, interm = 0.0
α = 2.5, new_cost = 616350.4061055984, L2fgrad = 1393.2788337114116, interm = 0.0
entered for loop, E=616350.4061055984, k = 4
α = 10, new_cost = 618798.197950853, L2fgrad = 713.32

(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction()), (80.0, SingleFieldFEFunction()), (120.0, SingleFieldFEFunction()), (160.0, SingleFieldFEFunction()), (200.0, SingleFieldFEFunction()), (240.0, SingleFieldFEFunction()), (280.0, SingleFieldFEFunction()), (320.0, SingleFieldFEFunction()), (360.0, SingleFieldFEFunction())  …  (28440.0, SingleFieldFEFunction()), (28480.0, SingleFieldFEFunction()), (28520.0, SingleFieldFEFunction()), (28560.0, SingleFieldFEFunction()), (28600.0, SingleFieldFEFunction()), (28640.0, SingleFieldFEFunction()), (28680.0, SingleFieldFEFunction()), (28720.0, SingleFieldFEFunction()), (28760.0, SingleFieldFEFunction()), (28800.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction()), (80.0, SingleFieldFEFunction()), (120.0, SingleFieldFEFunction()), (160.0, SingleFieldFE

In [51]:
price = get_price_function(price_NOK, tF, 1)

# c1_real = (plot_costs(costs3))
c1_real = (plot_costs2(costs3))

CairoMakie.save("price_real_convergence_beka.png", c1_real)

Q_int = get_control_int(qs3)
T_int = get_temperature_int(Ts3)
c2_real = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_real_result_beka.png", c2_real)


CairoMakie.Screen{IMAGE}


In [49]:
#Dette ga ikke veldig realistiske tall... hmmm

real_price = 0
for (t,QQ) in qs3
    real_price+=Δt*price(t)*∑(∫(QQ)*dΩ)*(1/1000/3600)#*(∑(∫(q_pos)*dΩ))
end
temp_difference =  0
tmp = last(Ts3)[2]-Tfin
temp_difference = ∑(∫(tmp)*dΩ)
println(real_price) 
println(temp_difference/(width*height))

0.11195937124249278
-0.8810225208162611


In [16]:
# # Opprett en mappe for midlertidige filer hvis den ikke eksisterer
# if !isdir("tmp")
#     mkdir("tmp")
# end

# # Initialtilstanden og resten av tidsløsningene
# # uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
# # uh  = solution[2:end]  # De resterende løsningene


# uh = Ts3
# # Lagre resultater i Paraview-format (VTK)
# createpvd("resultsOutisde") do pvd
#     # Første løsning (t=0)
#     # pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

#     # Lagrer løsninger for hver tidssteg
#     count = 0
#     for (tn, uhn) in uh
#         if count%2 == 0
#             pvd[tn] = createvtk(Ω, "tmp/results_$count.vtu", cellfields=["u" => uhn])
#         end
#         count+=1
#     end
# end
